<a href="https://colab.research.google.com/github/n0k0m3/rnn-mal-sentiment/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>